In [1]:
%cd /ibex/user/slimhy/PADS/code/

/ibex/user/slimhy/PADS/code


In [2]:
# Add "3DCoMPaT" to the path
import sys
sys.path.append("jupyter/3DCoMPaT/")
from compat3D import ShapeLoader

In [3]:
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import numpy as np
import torch

import util.misc as misc
import models.autoencoders as ae_mods

In [4]:
def get_args_parser():
    parser = argparse.ArgumentParser("Extracting Features", add_help=False)

    # Model parameters
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--intensity_loss",
        action="store_true",
        default=False,
        help="Contrastive edit intensity loss using ground-truth labels.",
    )

    # Dataset parameters
    parser.add_argument(
        "--dataset",
        type=str,
        choices=["graphedits"],
        help="dataset name",
    )
    parser.add_argument(
        "--data_path",
        type=str,
        help="dataset path",
    )
    parser.add_argument(
        "--data_type",
        type=str,
        help="dataset type",
    )
    parser.add_argument(
        "--max_edge_level",
        default=None,
        type=int,
        help="maximum edge level to use",
    )
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.",
    )

    return parser

In [5]:
# Set dummy arg string to debug the parser
call_string = """--ae_pth ckpt/ae_m512.pth \
    --ae kl_d512_m512_l8 \
    --ae-latent-dim 4096 \
    --batch_size 32 \
    --num_workers 8 \
    --device cuda"""
    

# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())

In [6]:
# --------------------
device = torch.device(args.device)

# Fix the seed for reproducibility
misc.set_all_seeds(args.seed)

torch.backends.cudnn.benchmark = True
# --------------------

# Instantiate autoencoder
ae = ae_mods.__dict__[args.ae]()
ae.eval()
print("Loading autoencoder %s" % args.ae_pth)
ae.load_state_dict(torch.load(args.ae_pth, map_location="cpu")["model"])
ae = ae.to(device)

# Compile using torch.compile
ae = torch.compile(ae, mode="max-autotune")

Set seed to 0
Loading autoencoder ckpt/ae_m512.pth


In [7]:
import mcubes
import trimesh


def get_grid(grid_density=128):
    x = np.linspace(-1, 1, grid_density+1)
    y = np.linspace(-1, 1, grid_density+1)
    z = np.linspace(-1, 1, grid_density+1)
    xv, yv, zv = np.meshgrid(x, y, z)
    grid = torch.from_numpy(np.stack([xv, yv, zv]).astype(np.float32)).view(3, -1).transpose(0, 1)[None].to(device, non_blocking=True)
    return grid


@torch.inference_mode()
def query_latents_from_set(latent, point_queries, batch_size=None):
    num_samples = point_queries.shape[1]

    if batch_size is None:
        logits = ae.decode(latent, point_queries).detach()
        return logits

    logits = torch.cat([
        ae.decode(latent, point_queries[:, start_idx:end_idx, :]).detach().cpu()
        for start_idx, end_idx in batch_slices(num_samples, batch_size)
    ], dim=1)

    return logits


@torch.inference_mode()
def query_latents(latent, grid_density, batch_size=None):
    sample_grid = get_grid(grid_density)
    return query_latents_from_set(latent, sample_grid, batch_size)


@torch.inference_mode()
def decode_latents(latent, grid_density=128, batch_size=None):
    logits = query_latents(latent, grid_density, batch_size)
    volume = logits.view(grid_density+1, grid_density+1, grid_density+1).permute(1, 0, 2).cpu().numpy()
    verts, faces = mcubes.marching_cubes(volume, 0)
    gap = 2. / grid_density
    verts *= gap
    verts -= 1
    return trimesh.Trimesh(verts, faces)


def batch_slices(total, batch_size):
    """
    Generate batch start and end indices.
    """
    start = 0
    while start < total:
        end = min(start + batch_size, total)
        yield start, end
        start = end


@torch.inference_mode()
def encode_pc(pc):
    subsample__pc = pc[np.random.choice(pc.shape[0], ae.num_inputs, replace=False)] 
    _, x_a = ae.encode(torch.from_numpy(subsample__pc).to(device).unsqueeze(0).float())
    return x_a


@torch.inference_mode()
def encode_decode(pc, grid_density=128, batch_size=None):
    x_a = encode_pc(pc)
    return decode_latents(x_a, grid_density, batch_size), x_a


In [8]:
import zipfile
from metadata import SHAPENET_CLASSES, COMPAT_CLASSES, COMPAT_TRANSFORMS


ACTIVE_CLASS = "chair"
METADATA_DIR = "/ibex/user/slimhy/3DCoMPaT/3DCoMPaT-v2/metadata"
ZIP_SRC = "/ibex/user/slimhy/surfaces.zip"
ZIP_PATH = "/ibex/user/slimhy/3DCoMPaT/3DCoMPaT_ZIP.zip"
N_POINTS = 2**21


def shapenet_iterator(shape_cls):
    # List all files in the zip file
    with zipfile.ZipFile(ZIP_SRC, 'r') as zip_ref:
        files = zip_ref.namelist()

        for file in files:
            if not file.startswith(SHAPENET_CLASSES[shape_cls]): continue
            if not file.endswith(".npz"): continue
            # Read a specific file from the zip file
            with zip_ref.open(file) as file:
                data = np.load(file)
                yield data["points"].astype(np.float32)
        

def compat_iterator(shape_cls):
    train_dataset = ShapeLoader(zip_path=ZIP_PATH,
                                meta_dir=METADATA_DIR,
                                split="train",
                                n_points=N_POINTS,
                                shuffle=True,
                                seed=0,
                                filter_class=COMPAT_CLASSES[shape_cls])

    for shape_id, shape_label, pointcloud, point_part_labels in train_dataset:
        yield COMPAT_TRANSFORMS[shape_cls](pointcloud)

In [9]:
from shapeloaders import SingleManifoldDataset

OBJ_DIR = "/ibex/user/slimhy/PADS/data/obj_manifold/"
OBJ_ID = 0

# Freeze the autoencoder
ae.eval()

# Instantiate the dataset
shape_dataset = SingleManifoldDataset(OBJ_DIR,
                                      ACTIVE_CLASS,
                                      N_POINTS,
                                      normalize=False,
                                      sampling_method="volume+near_surface",
                                      contain_method="occnets",
                                      decimate=True,
                                      sample_first=False)
it_shape = shape_dataset[OBJ_ID]

# Initialize the latents
orig_dataset = SingleManifoldDataset(OBJ_DIR,
                                     ACTIVE_CLASS,
                                     N_POINTS,
                                     normalize=False,
                                     sampling_method="surface")
surface_points, _ = next(orig_dataset[OBJ_ID])
init_latents = encode_pc(surface_points).detach()

In [10]:
from schedulefree import AdamWScheduleFree
import torch.nn.functional as F


def query_latents_with_graph(ae, latent, point_queries, batch_size=None):
    num_samples = point_queries.shape[1]

    if batch_size is None:
        logits = ae.decode(latent, point_queries)
        return logits

    logits = torch.cat([
        ae.decode(latent, point_queries[:, start_idx:end_idx, :]).detach().cpu()
        for start_idx, end_idx in batch_slices(num_samples, batch_size)
    ], dim=1)

    return logits


# Optimize the latents for a single object
def optimize_latents(ae, shape_it, init_latents, max_iter=100, optimizer=None):
    # Optimize the latents
    latents = torch.tensor(init_latents).to(device).requires_grad_(True)
    if optimizer is None:
        optimizer = torch.optim.Adam
    optimizer = optimizer([latents], lr=1e-3)
    
    # Optimize the latents
    for i, (surface_points, occs) in enumerate(shape_it):
        optimizer.zero_grad()
        # 1: Sample points from the shape surface
        surface_points = torch.tensor(surface_points).float().to(device).unsqueeze(0)
        
        # 2: Query the autoencoder
        logits = query_latents_with_graph(ae, latents, surface_points.to(device).float()).squeeze()
        occs = torch.tensor(occs).float().to(device)

        # 3: Compute BCE loss
        loss = F.binary_cross_entropy_with_logits(logits, occs).mean()
        
        # Add gradient clipping
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print("Iter %d: Loss %.4f" % (i, loss.item()))
        if i >= max_iter:
            break

    return latents.detach().cpu()

optimized_latents = None

def optimize_fn():
    global optimized_latents
    optimized_latents = optimize_latents(ae,
                                         it_shape,
                                         init_latents,
                                         max_iter=1500,
                                         optimizer=AdamWScheduleFree)


def speedtest(function_wrapper, exp_name):
    import cProfile
    import pstats

    with cProfile.Profile() as pr:
        function_wrapper()

    stats = pstats.Stats(pr)
    stats.sort_stats(pstats.SortKey.TIME)
    filename = "%s.prof" % exp_name
    stats.dump_stats(filename=filename)


optimize_fn()

/tmp/ipykernel_1045144/2232836724.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  latents = torch.tensor(init_latents).to(device).requires_grad_(True)


Iter 0: Loss 0.0285
Iter 10: Loss 0.0248
Iter 20: Loss 0.0229
Iter 30: Loss 0.0213
Iter 40: Loss 0.0202
Iter 50: Loss 0.0192
Iter 60: Loss 0.0188
Iter 70: Loss 0.0178
Iter 80: Loss 0.0170
Iter 90: Loss 0.0165
Iter 100: Loss 0.0163
Iter 110: Loss 0.0158
Iter 120: Loss 0.0157
Iter 130: Loss 0.0153
Iter 140: Loss 0.0152
Iter 150: Loss 0.0151
Iter 160: Loss 0.0148
Iter 170: Loss 0.0148
Iter 180: Loss 0.0145
Iter 190: Loss 0.0145
Iter 200: Loss 0.0144
Iter 210: Loss 0.0143
Iter 220: Loss 0.0142
Iter 230: Loss 0.0141
Iter 240: Loss 0.0140
Iter 250: Loss 0.0140
Iter 260: Loss 0.0138
Iter 270: Loss 0.0138
Iter 280: Loss 0.0136
Iter 290: Loss 0.0137
Iter 300: Loss 0.0135
Iter 310: Loss 0.0135
Iter 320: Loss 0.0135
Iter 330: Loss 0.0134
Iter 340: Loss 0.0133
Iter 350: Loss 0.0133
Iter 360: Loss 0.0132
Iter 370: Loss 0.0132
Iter 380: Loss 0.0133
Iter 390: Loss 0.0131
Iter 400: Loss 0.0130
Iter 410: Loss 0.0130
Iter 420: Loss 0.0130
Iter 430: Loss 0.0130
Iter 440: Loss 0.0129
Iter 450: Loss 0.0127

In [11]:
from eval.metrics import chamfer_reconstructed, iou_occupancies
from util.contains.inside_mesh import MeshIntersector


def detach_tensor(tensor):
    if isinstance(tensor, torch.Tensor):
        return tensor.detach().cpu().numpy()
    return tensor


@torch.inference_mode()
def evaluate_reconstruction(obj_original, rec_mesh, latents, n_queries):
    """
    Compute IoU and Chamfer distance between the original object and the reconstructed mesh.
    - IoU: Computed using 128^3 random points sampled from the 3D space.
    """
    # Sample N_POINTS points from obj_original and rec_mesh
    pc_original, _ = trimesh.sample.sample_surface_even(obj_original, N_POINTS)
    pc_rec, _ = trimesh.sample.sample_surface_even(rec_mesh, N_POINTS)

    pc_original = torch.tensor(pc_original).float().to(device).unsqueeze(0)
    pc_rec = torch.tensor(pc_rec).float().to(device).unsqueeze(0)

    # Compute all metrics
    with torch.no_grad():
        chamfer_dist = chamfer_reconstructed(pc_original, pc_rec)

        # Sample n_queries points from the 3D space
        point_queries = torch.rand((1, n_queries, 3)).cuda()
        
        # Predict occupancy for each point using the autoencoder
        latents = torch.tensor(latents).to(device)
        logits = query_latents_from_set(latents, point_queries, batch_size=128**3)

        # Apply sigmoid to logits to get occupancy probabilities
        logits = F.sigmoid(logits)
        pred_occs = detach_tensor((logits > 0.5).reshape(n_queries))

        # Get ground-truth occupancy for each point
        intersector = MeshIntersector(obj_original, resolution=512)
        gt_occs = intersector.query(detach_tensor(point_queries.squeeze()))

        # Compute IoU between predicted and ground-truth occupancies
        iou = iou_occupancies(pred_occs, gt_occs)
    
    return iou, chamfer_dist.item()

Jitting Chamfer 3D
Loaded JIT 3D CUDA chamfer distance


In [12]:
# Decode the optimized latents
rec_mesh = decode_latents(torch.tensor(optimized_latents).to(device), grid_density=256, batch_size=128**3)
rec_mesh.show()

In [13]:
orig_mesh = shape_dataset.obj
evaluate_reconstruction(orig_mesh, rec_mesh, optimized_latents, 2**15)

(0.9836956530600188, 7.561196980532259e-06)

In [14]:
# Decode the old latents
old_latents = init_latents
old_rec_mesh = decode_latents(torch.tensor(old_latents).to(device), grid_density=256, batch_size=128**3)

evaluate_reconstruction(orig_mesh, old_rec_mesh, old_latents, 2**15)

(0.9408867024193742, 5.834340299770702e-06)

In [15]:
old_rec_mesh.show()